In [1]:
import unicodecsv
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

### Dataframe 1

In [2]:
df_product_id = pd.read_csv('model/product_id_md2.csv')
df_product_id.head()

,MATNR,Style,Cat2,Style_id,Cat2_id
0,19047936,Vintage,ตู้ข้างเตียง,25,61
1,19045101,Modern,โซฟา 3 ที่นั่ง,12,203
2,19050282,Vintage,กรอบรูป,25,5
3,59000365,Contemporary,ชุดห้องนอน,5,51
4,59001810,Modern,ชุดห้องนอน,12,51


### Dataframe 2

In [3]:
df_rgb = pd.read_csv('model/rgb_item.csv')
df_rgb.head()

,MATNR,R,G,B
0,19047936,255.0,255.0,255.0
1,19045101,128.0,115.0,38.0
2,19050282,140.0,110.0,58.0
3,59000365,38.0,31.0,30.0
4,59001810,245.0,245.0,245.0


### Merge Dataframe

In [4]:
df_product_rgb = pd.merge(df_product_id, df_rgb,  how='left', left_on=['MATNR'], right_on = ['MATNR'])
df_product_rgb.head()

,MATNR,Style,Cat2,Style_id,Cat2_id,R,G,B
0,19047936,Vintage,ตู้ข้างเตียง,25,61,255.0,255.0,255.0
1,19045101,Modern,โซฟา 3 ที่นั่ง,12,203,128.0,115.0,38.0
2,19050282,Vintage,กรอบรูป,25,5,140.0,110.0,58.0
3,59000365,Contemporary,ชุดห้องนอน,5,51,38.0,31.0,30.0
4,59001810,Modern,ชุดห้องนอน,12,51,245.0,245.0,245.0


# Model 2.1 Color Detection

### Normal Scale 

In [5]:
from sklearn.neighbors import NearestNeighbors

input_item = 19117034  #ใส่ input(MATNR)
input_index = df_product_rgb.index[df_product_rgb['MATNR'] == input_item].tolist()[0]
call_frame = df_product_rgb.loc[(df_product_rgb.MATNR.isin([input_item])), ['Style_id', 'Cat2_id', 'R', 'G', 'B']]
item = call_frame.values

product = df_product_rgb.iloc[:,3:8].values
nbrs = NearestNeighbors(n_neighbors=11).fit(product)
nbrs_res = nbrs.kneighbors([item[0]])[1]
nbrs_list = nbrs_res.tolist()[0]
while input_index in nbrs_list: nbrs_list.remove(input_index)
result = nbrs_list
#print(result)
df_product_rgb.iloc[result]

,MATNR,Style,Cat2,Style_id,Cat2_id,R,G,B
27545,19132830,Modern,โต๊ะอาหารขาเหล็กท๊อปไม้,12,244,131.0,136.0,150.0
6220,19085392,Modern,โต๊ะอาหารขาเหล็กท๊อปกระจก,12,242,137.0,136.0,140.0
9078,19097468,Modern,โต๊ะอาหารขาเหล็กท๊อปกระจก,12,242,140.0,140.0,140.0
6323,19089149,Modern,โต๊ะอาหารขาเหล็กท๊อปกระจก,12,242,141.0,142.0,141.0
4782,59003555,Modern,โต๊ะอาหารขาไม้ท๊อปกระจก,12,245,143.0,140.0,141.0
16446,59006871,Modern,โต๊ะทำงาน,12,235,141.0,141.0,141.0
26322,19112052,Modern,โต๊ะอาหารไม้ล้วน,12,251,131.0,133.0,136.0
12979,19060035,Modern,ไซด์บอร์ด,12,257,141.0,136.0,146.0
19289,59002866,Modern,โต๊ะอาหารขาเหล็กท๊อปกระจก,12,242,144.0,136.0,139.0
6747,59004528,Modern,โต๊ะอาหารขาเหล็กท๊อปหิน,12,243,146.0,145.0,146.0


Rescale(Normalize)

In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import normalize

sty_id_norm = df_product_rgb[['Style_id']].values
min_max_scaler = preprocessing.MinMaxScaler()
sty_scaled = min_max_scaler.fit_transform(sty_id_norm)
df_sty_norm = pd.DataFrame(sty_scaled, columns=['Style_nid'])

cat2_id_norm = df_product_rgb[['Cat2_id']].values
cat2_scaled = min_max_scaler.fit_transform(cat2_id_norm)
df_cat2_norm = pd.DataFrame(cat2_scaled, columns=['Cat2_nid'])

r_norm = df_product_rgb[['R']].values
r_scaled = min_max_scaler.fit_transform(r_norm)
df_r_norm = pd.DataFrame(r_scaled, columns=['R_n'])

g_norm = df_product_rgb[['G']].values
g_scaled = min_max_scaler.fit_transform(g_norm)
df_g_norm = pd.DataFrame(g_scaled, columns=['G_n'])

b_norm = df_product_rgb[['B']].values
b_scaled = min_max_scaler.fit_transform(b_norm)
df_b_norm = pd.DataFrame(b_scaled, columns=['B_n'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Add DFn

In [7]:
product = pd.read_csv('../../2_data/explored/product.csv')
color = pd.read_csv('../../2_data/explored/md6t_catcolor.csv')
product.drop(product.columns[[0,2,3,4,5,6,7,9,10,11]], axis=1, inplace = True)
product_color = pd.merge(product, color,  how='left', left_on=['MVGR6'], right_on = ['NODE'])
df_color_raw = product_color[['LTEXT']]
df_color_raw.rename(columns={'LTEXT': 'Color'}, inplace=True)
df_product_rgb_n = pd.concat([df_product_rgb, df_color_raw, df_sty_norm, df_cat2_norm, df_r_norm, df_g_norm, df_b_norm], axis=1) 
df_product_rgb_n.drop(df_product_rgb_n.columns[[3,4,5,6,7]], axis=1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


# Model 2.2 Color Detection

### Rescale

In [8]:
from sklearn.neighbors import NearestNeighbors

input_item = 19117034 #ใส่ input(MATNR)
input_index = df_product_rgb_n.index[df_product_rgb_n['MATNR'] == input_item].tolist()[0]
call_frame = df_product_rgb_n.loc[(df_product_rgb_n.MATNR.isin([input_item])), ['Style_nid', 'Cat2_nid', 'R_n', 'G_n', 'B_n']]
item = call_frame.values

product = df_product_rgb_n.iloc[:,4:9].values
nbrs = NearestNeighbors(n_neighbors=11).fit(product)
nbrs_res = nbrs.kneighbors([item[0]])[1]
nbrs_list = nbrs_res.tolist()[0]
while input_index in nbrs_list: nbrs_list.remove(input_index)
result = nbrs_list
#print(result)
df_product_rgb_n.iloc[result]

,MATNR,Style,Cat2,Color,Style_nid,Cat2_nid,R_n,G_n,B_n
27545,19132830,Modern,โต๊ะอาหารขาเหล็กท๊อปไม้,DARK SHERWOOD,0.036066,0.452514,0.244403,0.253731,0.279851
6220,19085392,Modern,โต๊ะอาหารขาเหล็กท๊อปกระจก,BLACK,0.036066,0.448790,0.255597,0.253731,0.261194
9078,19097468,Modern,โต๊ะอาหารขาเหล็กท๊อปกระจก,STAINLESS,0.036066,0.448790,0.261194,0.261194,0.261194
6323,19089149,Modern,โต๊ะอาหารขาเหล็กท๊อปกระจก,BLACK,0.036066,0.448790,0.263060,0.264925,0.263060
4782,59003555,Modern,โต๊ะอาหารขาไม้ท๊อปกระจก,DARK SHADE,0.036066,0.454376,0.266791,0.261194,0.263060
16446,59006871,Modern,โต๊ะทำงาน,WHITE SHADE,0.036066,0.435754,0.263060,0.263060,0.263060
26322,19112052,Modern,โต๊ะอาหารไม้ล้วน,GREY,0.036066,0.465549,0.244403,0.248134,0.253731
12979,19060035,Modern,ไซด์บอร์ด,GREY TWIST,0.036066,0.476723,0.263060,0.253731,0.272388
19289,59002866,Modern,โต๊ะอาหารขาเหล็กท๊อปกระจก,DARK SHADE,0.036066,0.448790,0.268657,0.253731,0.259328
6747,59004528,Modern,โต๊ะอาหารขาเหล็กท๊อปหิน,LIGHT SHADE,0.036066,0.450652,0.272388,0.270522,0.272388


Check Item

In [9]:
df_product_rgb_n.loc[df_product_rgb_n['MATNR'] == 19117034 ]

,MATNR,Style,Cat2,Color,Style_nid,Cat2_nid,R_n,G_n,B_n
22915,19117034,Modern,โต๊ะอาหารขาเหล็กท๊อปไม้,DARK SHERWOOD,0.036066,0.452514,0.246269,0.255597,0.279851
